# 01. Perform simulations

In [7]:
import simserialED.core as simserialED
from orix.crystal_map import Phase
from orix.sampling import get_sample_reduced_fundamental, get_sample_fundamental
from diffsims.generators.simulation_generator import SimulationGenerator
from diffpy.structure import Lattice, Structure, Atom

In [18]:
# from parse_pdb_with_scale import parse_pdb_with_scale
from parse_pdb_with_scale_remove_h import parse_pdb_with_scale
pdbfile = "/Users/xiaodong/Desktop/UOXsim-3/UOX.pdb"

# cell, sg_sym, atoms = parse_pdb_with_scale(pdbfile)
cell, sg_sym, atoms = parse_pdb_with_scale(pdbfile, remove_hydrogens=False, include_occupancy=True)

# print(cell)
# print(sg_sym)
# print(atoms)
print(len(atoms))

lattice = Lattice(*cell)

# Create the structure and corresponding phase
structure = Structure(atoms, lattice)

print(structure)


5163
lattice=Lattice(a=80.58, b=94.49, c=103.89, alpha=90, beta=90, gamma=90)
N    0.761267 0.612883 0.197468 1.0000
C    0.745345 0.606268 0.201337 0.3000
C    0.731272 0.616566 0.198738 0.3000
O    0.733903 0.628069 0.193540 0.3000
C    0.745767 0.602152 0.215565 0.3000
O    0.757990 0.591643 0.217798 0.3000
H    0.766578 0.607200 0.192019 0.3000
H    0.759269 0.621137 0.193646 0.3000
H    0.767534 0.614290 0.204408 0.3000
H    0.743446 0.597654 0.196467 0.3000
H    0.748385 0.610459 0.220657 0.3000
H    0.734995 0.598469 0.218029 0.3000
H    0.758251 0.589526 0.225643 0.3000
C    0.745978 0.608470 0.204552 0.7000
C    0.731346 0.618026 0.201289 0.7000
O    0.733630 0.630292 0.197718 0.7000
C    0.749080 0.608724 0.219059 0.7000
O    0.761279 0.598606 0.222466 0.7000
H    0.763463 0.606639 0.191288 0.7000
H    0.759678 0.621381 0.193974 0.7000
H    0.769780 0.613274 0.202916 0.7000
H    0.743123 0.598881 0.202030 0.7000
H    0.752915 0.618068 0.221523 0.7000
H    0.738817 0.606501 0.

In [ ]:
from parse_pdb_with_scale_remove_h import parse_pdb_with_scale

pdbfile = "/Users/xiaodong/Desktop/UOX-simulations/UOX.pdb"

# cell, sg_sym, atoms = parse_pdb_with_scale(pdbfile)
cell, sg_sym, atoms = parse_pdb_with_scale(pdbfile, remove_hydrogens=True, include_occupancy=True)

lattice = Lattice(*cell)


# Define the phase using less atoms than in the pdb for computational reasons
atoms = [
         Atom("C", (0, 0, 0), 1.0),
        #  Atom("H", (0.245209, 0.481537, 0.392577), 1.0),
        #  Atom("C", (0.505211, 0.153453, 0.324897), 1.0),
        #  Atom("N", (0.761267, 0.612883, 0.197468), 1.0),
        #  Atom("O", (0.186473, 0.433808, 0.369504), 1.0),
        #  Atom("S", (0.344402, 0.376522, 0.265119), 0.41)
         ]

# Create the structure and corresponding phase
structure = Structure(atoms, lattice)
phase = Phase(space_group = 23, structure=structure)

# Sample orientations in the symmetry-reduced zone (resolution in degrees)
orientations = get_sample_reduced_fundamental(
    resolution=10,
    point_group=phase.point_group,
)

# Initialize the simulation generator (e.g., setting the accelerating voltage)
generator = SimulationGenerator(
        accelerating_voltage = 300,
        scattering_params = "lobato",
        precession_angle = 0,
        shape_factor_model = "lorentzian",
        approximate_precession = False,
        minimum_intensity = 1e-20,
        )

# Calculate 2D diffraction simulations using the provided phase and orientations
simulations = generator.calculate_diffraction2d(
    phase = phase,
    rotation = orientations,
    reciprocal_radius = 1/1.3,  # 1/Å (i.e. Å⁻¹)
    with_direct_beam = True,   # Include the direct beam
    max_excitation_error = 0.0005,
    shape_factor_width = None,
    debye_waller_factors = None,
    show_progressbar = True
)

# Plot the simulations (this may include an interactive slider)
simulations.plot(
        size_factor=1,
        direct_beam_position=None,
        in_plane_angle=0,
        mirrored=False,
        units="real",
        show_labels=False,
        label_offset=(0, 0),
        label_formatting=None,
        min_label_intensity=0.1,
        include_direct_beam=True,
        calibration = 0.0015090274190359715,
        ax=None,
        interactive=False,
    )

100%|██████████| 171/171 [00:11<00:00, 15.42it/s]


'Calculates the diffraction pattern for one or more phases given a list\nof rotations for each phase.\n\nParameters\n----------\nphase:\n    The phase(s) for which to derive the diffraction pattern.\nreciprocal_radius\n    The maximum radius of the sphere of reciprocal space to\n    sample, in reciprocal Angstroms.\nrotation\n    The Rotation object(s) to apply to the structure and then\n    calculate the diffraction pattern.\nwith_direct_beam\n    If True, the direct beam is included in the simulated\n    diffraction pattern. If False, it is not.\nmax_excitation_error\n    The cut-off for geometric excitation error in the z-direction\n    in units of reciprocal Angstroms. Spots with a larger distance\n    from the Ewald sphere are removed from the pattern.\n    Related to the extinction distance and roughly equal to 1/thickness.\nshape_factor_width\n    Determines the width of the reciprocal rel-rod, for fine-grained\n    control. If not set will be set equal to max_excitation_error.\

# 03. Add simulations to dataset

In [86]:
import shutil
import h5py
import random
from tqdm import tqdm
import os

# Input (original) file
input_filename = "/Users/xiaodong/Desktop/UOX-simulations/UOX1_sub_radial_backgrounds.h5"

# Get the directory of the input file
base_dir = os.path.dirname(input_filename)

# Find the next available simulation folder (simulation-1, simulation-2, etc.)
folder_number = 1
while os.path.exists(os.path.join(base_dir, f"simulation-{folder_number}")):
    folder_number += 1

# Create the new simulation folder
new_simulation_folder = os.path.join(base_dir, f"simulation-{folder_number}")
os.makedirs(new_simulation_folder)

# Define the output file path within the new simulation folder
output_filename = os.path.join(new_simulation_folder, "sim.h5")

# Make an identical copy of the original file
shutil.copyfile(input_filename, output_filename)

# Open the copy in read/write mode so that you can modify it safely
with h5py.File(output_filename, "r+") as data:
    images = data["entry"]["data"]["images"]

    # Create (or open if they already exist) a dataset to store simulation Euler angles.
    euler_angles = data["entry"]["data"].require_dataset(
        "simulation_euler_angles", 
        shape=(images.shape[0], 3), 
        dtype=float
    )

    # Get the image shape (assumes the last two dimensions hold the 2D image)
    shape = images.shape[-2:]
    calibration = 0.0015090274190359715 # calibration: 1/Å per pixel
    intensity_scale = 1000

    # Loop over each image using a progress bar
    for i in tqdm(range(images.shape[0]), desc="Processing images"):
        # Choose a random in-plane rotation angle (in degrees)
        in_plane = random.uniform(0, 360)

        # Choose a random simulation index
        ind = random.randint(0, simulations.current_size - 1)

        # Generate simulation diffraction image using the chosen simulation and rotation
        img = simulations.irot[ind].get_diffraction_pattern(
            shape=shape,
            direct_beam_position=(shape[0] // 2, shape[1] // 2),  # pixel coordinates of the direct beam
            sigma=1.5,
            in_plane_angle=in_plane,
            calibration=calibration,
            fast=False,
            normalize = True,
            fast_clip_threshold = 5 #0.1,
        )
        # /diffsims/simulations/simulation2d.py
        """
        shape  : tuple of ints
            The size of a side length (in pixels)
        sigma : float
            Standard deviation of the Gaussian function to be plotted (in pixels).
        direct_beam_position: 2-tuple of ints, optional
            The (x,y) coordinate in pixels of the direct beam. Defaults to
            the center of the image.
        in_plane_angle: float, optional
            In plane rotation of the pattern in degrees
        mirrored: bool, optional
            Whether the pattern should be flipped over the x-axis,
            corresponding to the inverted orientation
        fast: bool, optional
            Whether to speed up calculations by rounding spot coordinates down to integer pixel
        normalize: bool, optional
            Whether to normalize the pattern to values between 0 and 1
        fast_clip_threshold: float, optional
            Only used when `fast` is False.
            Pixel intensity threshold, such that pixels which would be below this value are ignored.
            Thresholding performed before possible normalization.
            See diffsims.pattern.detector_functions.get_pattern_from_pixel_coordinates_and_intensities for details.
        Returns
        """
        # Scale image intensity to match the original image's data type
        img = (img * intensity_scale).astype(images.dtype)

        # Option A: Directly add the simulation to the existing "images" dataset
        images[i] += img


        # Retrieve and adjust Euler angles from the simulation
        euler_angles_i = simulations.rotations[ind].to_euler(degrees=True).squeeze()
        euler_angles_i[0] = in_plane  # Override the first angle with the random in-plane rotation
        euler_angles[i] = euler_angles_i

print("Done! The combined file is saved at:", output_filename)

# Make sure your notebook uses an interactive backend.
# For inline static images, you can use:
%matplotlib inline
# For interactive figures, you can also try:
# %matplotlib widget

import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import clear_output

# Open the file and preload the datasets into memory.
filename = output_filename

with h5py.File(filename, "r") as hf:
    images = hf["entry"]["data"]["images"]
    euler_angles = hf["entry"]["data"]["simulation_euler_angles"]
    
    # Optional: print shapes for debugging
    # print("Images shape:", images.shape)
    # print("Euler angles shape:", euler_angles.shape)
    
    # Preload the data (this works well if the datasets aren't too large)
    images_arr = images[:]       # All images as a NumPy array.
    angles_arr = euler_angles[:]   # All Euler angles.

def view_image(index):
    # Clear the previous output for a smoother experience.
    clear_output(wait=True)
    
    # Retrieve the image and Euler angles for the selected index.
    img = images_arr[index]
    angles = angles_arr[index]
    
    plt.figure(figsize=(6, 6))
    plt.imshow(img, cmap="gray")
    plt.title(f"Image Index: {index}\nEuler Angles: {angles}")
    plt.axis("off")
    plt.show()

# Create an interactive slider.
interact(view_image, index=widgets.IntSlider(min=0, max=images_arr.shape[0]-1, step=1, value=0))

Processing images: 100%|██████████| 100/100 [00:02<00:00, 42.48it/s]


Done! The combined file is saved at: /Users/xiaodong/Desktop/UOX-simulations/simulation-19/sim.h5


interactive(children=(IntSlider(value=0, description='index', max=99), Output()), _dom_classes=('widget-intera…

<function __main__.view_image(index)>

# 05. Visualize simulations on dataset

In [73]:
# Make sure your notebook uses an interactive backend.
# For inline static images, you can use:
%matplotlib inline
# For interactive figures, you can also try:
# %matplotlib widget

import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import clear_output

# Open the file and preload the datasets into memory.
filename = output_filename

with h5py.File(filename, "r") as hf:
    images = hf["entry"]["data"]["images"]
    euler_angles = hf["entry"]["data"]["simulation_euler_angles"]
    
    # Optional: print shapes for debugging
    # print("Images shape:", images.shape)
    # print("Euler angles shape:", euler_angles.shape)
    
    # Preload the data (this works well if the datasets aren't too large)
    images_arr = images[:]       # All images as a NumPy array.
    angles_arr = euler_angles[:]   # All Euler angles.

def view_image(index):
    # Clear the previous output for a smoother experience.
    clear_output(wait=True)
    
    # Retrieve the image and Euler angles for the selected index.
    img = images_arr[index]
    angles = angles_arr[index]
    
    plt.figure(figsize=(6, 6))
    plt.imshow(img, cmap="gray")
    plt.title(f"Image Index: {index}\nEuler Angles: {angles}")
    plt.axis("off")
    plt.show()

# Create an interactive slider.
interact(view_image, index=widgets.IntSlider(min=0, max=images_arr.shape[0]-1, step=1, value=0))


interactive(children=(IntSlider(value=0, description='index', max=99), Output()), _dom_classes=('widget-intera…

<function __main__.view_image(index)>